# Aula4.Ex5 - Cilindro - Transformação Geométrica 3D

### Primeiro, importamos as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Cilindro", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Aprenderemos GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4). Use ela como matriz de transformação, resultante de uma sequências de outras transformações (e.g. rotação + translação).
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso combinamos nossa variável vec2 com uma variável vec4. Além disso, modificamos nosso vetor com base em uma matriz de transformação.

In [3]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal).
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, criaremos uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando um Cilindro

In [11]:
import math


PI = 3.141592
r = 0.5 # raio
H = 0.6
num_sectors = 50 # qtd de sectors (longitude)
num_stacks = 50 # qtd de stacks (latitude)

# grid sectos vs stacks (longitude vs latitude)
sector_step = (PI*2)/num_sectors # variar de 0 até 2π
stack_step = H/num_stacks # variar de 0 até H

# Entrada: angulo de t, altura h, raio r
# Saida: coordenadas no cilindro
def CoordCilindro(t, h, r):
    x = r * math.cos(t)
    y = r * math.sin(t)
    z = h
    return (x,y,z)

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos
vertices_list = []
for j in range(0,num_stacks): # para cada stack (latitude)
    
    for i in range(0,num_sectors): # para cada sector (longitude) 
        
        u = i * sector_step # angulo setor
        v = j * stack_step # altura da stack
        
        un = 0 # angulo do proximo sector
        if i+1==num_sectors:
            un = PI*2
        else: un = (i+1)*sector_step
            
        vn = 0 # altura da proxima stack
        if j+1==num_stacks:
            vn = H
        else: vn = (j+1)*stack_step
        
        # verticies do poligono
        p0=CoordCilindro(u, v, r)
        p1=CoordCilindro(u, vn, r)
        p2=CoordCilindro(un, v, r)
        p3=CoordCilindro(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_list.append(p0)
        vertices_list.append(p2)
        vertices_list.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_list.append(p3)
        vertices_list.append(p1)
        vertices_list.append(p2)
        
        print(v)
        
        if v == 0:
            vertices_list.append(p0)
            vertices_list.append(p2)
            vertices_list.append(CoordCilindro(0, v, 0))
        if vn == H:
            #faz um triangulo a partir do mesmo angulo u, mas com as alturas em h = vn
            vertices_list.append(p1)
            vertices_list.append(p3)
            vertices_list.append(CoordCilindro(0, vn, 0))

total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.012
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.024
0.036000000000000004
0.036000000000000004
0.036000000000000004
0.036000000000000004
0.036000000000000004
0.036000000000000004
0.036000000000000004
0.036000000000000004
0.036000000000000004
0.036000000

0.252
0.252
0.252
0.252
0.252
0.252
0.252
0.252
0.252
0.252
0.252
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.264
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.276
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.28800000000000003
0.288000000000

In [12]:
for v in vertices:
    print(v)

([0.5, 0. , 0. ],)
([0.49605736, 0.0626666 , 0.        ],)
([0.5  , 0.   , 0.012],)
([0.49605736, 0.0626666 , 0.012     ],)
([0.5  , 0.   , 0.012],)
([0.49605736, 0.0626666 , 0.        ],)
([0.5, 0. , 0. ],)
([0.49605736, 0.0626666 , 0.        ],)
([0., 0., 0.],)
([0.49605736, 0.0626666 , 0.        ],)
([0.48429158, 0.12434492, 0.        ],)
([0.49605736, 0.0626666 , 0.012     ],)
([0.48429158, 0.12434492, 0.012     ],)
([0.49605736, 0.0626666 , 0.012     ],)
([0.48429158, 0.12434492, 0.        ],)
([0.49605736, 0.0626666 , 0.        ],)
([0.48429158, 0.12434492, 0.        ],)
([0., 0., 0.],)
([0.48429158, 0.12434492, 0.        ],)
([0.46488824, 0.18406224, 0.        ],)
([0.48429158, 0.12434492, 0.012     ],)
([0.46488824, 0.18406224, 0.012     ],)
([0.48429158, 0.12434492, 0.012     ],)
([0.46488824, 0.18406224, 0.        ],)
([0.48429158, 0.12434492, 0.        ],)
([0.46488824, 0.18406224, 0.        ],)
([0., 0., 0.],)
([0.46488824, 0.18406224, 0.        ],)
([0.43815336, 0.2408768 

([-0.15450831,  0.47552833,  0.012     ],)
([-0.21288946,  0.45241362,  0.012     ],)
([-0.15450831,  0.47552833,  0.024     ],)
([-0.21288946,  0.45241362,  0.024     ],)
([-0.15450831,  0.47552833,  0.024     ],)
([-0.21288946,  0.45241362,  0.012     ],)
([-0.21288946,  0.45241362,  0.012     ],)
([-0.26791322,  0.42216408,  0.012     ],)
([-0.21288946,  0.45241362,  0.024     ],)
([-0.26791322,  0.42216408,  0.024     ],)
([-0.21288946,  0.45241362,  0.024     ],)
([-0.26791322,  0.42216408,  0.012     ],)
([-0.26791322,  0.42216408,  0.012     ],)
([-0.31871182,  0.38525677,  0.012     ],)
([-0.26791322,  0.42216408,  0.024     ],)
([-0.31871182,  0.38525677,  0.024     ],)
([-0.26791322,  0.42216408,  0.024     ],)
([-0.31871182,  0.38525677,  0.012     ],)
([-0.31871182,  0.38525677,  0.012     ],)
([-0.36448413,  0.34227374,  0.012     ],)
([-0.31871182,  0.38525677,  0.024     ],)
([-0.36448413,  0.34227374,  0.024     ],)
([-0.31871182,  0.38525677,  0.024     ],)
([-0.364484

([0.21288975, 0.45241347, 0.036     ],)
([0.2679135, 0.4221639, 0.048    ],)
([0.21288975, 0.45241347, 0.048     ],)
([0.2679135, 0.4221639, 0.048    ],)
([0.21288975, 0.45241347, 0.036     ],)
([0.21288975, 0.45241347, 0.036     ],)
([0.15450862, 0.4755282 , 0.036     ],)
([0.21288975, 0.45241347, 0.048     ],)
([0.15450862, 0.4755282 , 0.048     ],)
([0.21288975, 0.45241347, 0.048     ],)
([0.15450862, 0.4755282 , 0.036     ],)
([0.15450862, 0.4755282 , 0.036     ],)
([0.0936908 , 0.49114358, 0.036     ],)
([0.15450862, 0.4755282 , 0.048     ],)
([0.0936908 , 0.49114358, 0.048     ],)
([0.15450862, 0.4755282 , 0.048     ],)
([0.0936908 , 0.49114358, 0.036     ],)
([0.0936908 , 0.49114358, 0.036     ],)
([0.03139542, 0.49901336, 0.036     ],)
([0.0936908 , 0.49114358, 0.048     ],)
([0.03139542, 0.49901336, 0.048     ],)
([0.0936908 , 0.49114358, 0.048     ],)
([0.03139542, 0.49901336, 0.036     ],)
([0.03139542, 0.49901336, 0.036     ],)
([-0.03139509,  0.49901336,  0.036     ],)
([0

([ 0.464888  , -0.18406285,  0.06      ],)
([ 0.48429143, -0.12434555,  0.06      ],)
([ 0.464888  , -0.18406285,  0.06      ],)
([ 0.48429143, -0.12434555,  0.048     ],)
([ 0.48429143, -0.12434555,  0.048     ],)
([ 0.49605727, -0.06266725,  0.048     ],)
([ 0.48429143, -0.12434555,  0.06      ],)
([ 0.49605727, -0.06266725,  0.06      ],)
([ 0.48429143, -0.12434555,  0.06      ],)
([ 0.49605727, -0.06266725,  0.048     ],)
([ 0.49605727, -0.06266725,  0.048     ],)
([ 5.000000e-01, -6.535898e-07,  4.800000e-02],)
([ 0.49605727, -0.06266725,  0.06      ],)
([ 5.000000e-01, -6.535898e-07,  6.000000e-02],)
([ 0.49605727, -0.06266725,  0.06      ],)
([ 5.000000e-01, -6.535898e-07,  4.800000e-02],)
([0.5 , 0.  , 0.06],)
([0.49605736, 0.0626666 , 0.06      ],)
([0.5  , 0.   , 0.072],)
([0.49605736, 0.0626666 , 0.072     ],)
([0.5  , 0.   , 0.072],)
([0.49605736, 0.0626666 , 0.06      ],)
([0.49605736, 0.0626666 , 0.06      ],)
([0.48429158, 0.12434492, 0.06      ],)
([0.49605736, 0.062666

([-0.4960574 , -0.06266628,  0.084     ],)
([-0.48429167, -0.1243446 ,  0.084     ],)
([-0.4960574 , -0.06266628,  0.084     ],)
([-0.48429167, -0.1243446 ,  0.072     ],)
([-0.48429167, -0.1243446 ,  0.072     ],)
([-0.46488836, -0.18406193,  0.072     ],)
([-0.48429167, -0.1243446 ,  0.084     ],)
([-0.46488836, -0.18406193,  0.084     ],)
([-0.48429167, -0.1243446 ,  0.084     ],)
([-0.46488836, -0.18406193,  0.072     ],)
([-0.46488836, -0.18406193,  0.072     ],)
([-0.43815354, -0.24087651,  0.072     ],)
([-0.46488836, -0.18406193,  0.084     ],)
([-0.43815354, -0.24087651,  0.084     ],)
([-0.46488836, -0.18406193,  0.084     ],)
([-0.43815354, -0.24087651,  0.072     ],)
([-0.43815354, -0.24087651,  0.072     ],)
([-0.40450874, -0.2938923 ,  0.072     ],)
([-0.43815354, -0.24087651,  0.084     ],)
([-0.40450874, -0.2938923 ,  0.084     ],)
([-0.43815354, -0.24087651,  0.084     ],)
([-0.40450874, -0.2938923 ,  0.072     ],)
([-0.40450874, -0.2938923 ,  0.072     ],)
([-0.364484

...

([-0.03139509,  0.49901336,  0.528     ],)
([-0.09369048,  0.49114367,  0.528     ],)
([-0.03139509,  0.49901336,  0.528     ],)
([-0.09369048,  0.49114367,  0.516     ],)
([-0.09369048,  0.49114367,  0.516     ],)
([-0.15450831,  0.47552833,  0.516     ],)
([-0.09369048,  0.49114367,  0.528     ],)
([-0.15450831,  0.47552833,  0.528     ],)
([-0.09369048,  0.49114367,  0.528     ],)
([-0.15450831,  0.47552833,  0.516     ],)
([-0.15450831,  0.47552833,  0.516     ],)
([-0.21288946,  0.45241362,  0.516     ],)
([-0.15450831,  0.47552833,  0.528     ],)
([-0.21288946,  0.45241362,  0.528     ],)
([-0.15450831,  0.47552833,  0.528     ],)
([-0.21288946,  0.45241362,  0.516     ],)
([-0.21288946,  0.45241362,  0.516     ],)
([-0.26791322,  0.42216408,  0.516     ],)
([-0.21288946,  0.45241362,  0.528     ],)
([-0.26791322,  0.42216408,  0.528     ],)
([-0.21288946,  0.45241362,  0.528     ],)
([-0.26791322,  0.42216408,  0.516     ],)
([-0.26791322,  0.42216408,  0.516     ],)
([-0.318711

([0.15450862, 0.4755282 , 0.54      ],)
([0.0936908 , 0.49114358, 0.54      ],)
([0.15450862, 0.4755282 , 0.552     ],)
([0.0936908 , 0.49114358, 0.552     ],)
([0.15450862, 0.4755282 , 0.552     ],)
([0.0936908 , 0.49114358, 0.54      ],)
([0.0936908 , 0.49114358, 0.54      ],)
([0.03139542, 0.49901336, 0.54      ],)
([0.0936908 , 0.49114358, 0.552     ],)
([0.03139542, 0.49901336, 0.552     ],)
([0.0936908 , 0.49114358, 0.552     ],)
([0.03139542, 0.49901336, 0.54      ],)
([0.03139542, 0.49901336, 0.54      ],)
([-0.03139509,  0.49901336,  0.54      ],)
([0.03139542, 0.49901336, 0.552     ],)
([-0.03139509,  0.49901336,  0.552     ],)
([0.03139542, 0.49901336, 0.552     ],)
([-0.03139509,  0.49901336,  0.54      ],)
([-0.03139509,  0.49901336,  0.54      ],)
([-0.09369048,  0.49114367,  0.54      ],)
([-0.03139509,  0.49901336,  0.552     ],)
([-0.09369048,  0.49114367,  0.552     ],)
([-0.03139509,  0.49901336,  0.552     ],)
([-0.09369048,  0.49114367,  0.54      ],)
([-0.09369048

([0.40450853, 0.29389256, 0.576     ],)
([0.43815336, 0.2408768 , 0.576     ],)
([0.40450853, 0.29389256, 0.564     ],)
([0.40450853, 0.29389256, 0.564     ],)
([0.36448437, 0.3422735 , 0.564     ],)
([0.40450853, 0.29389256, 0.576     ],)
([0.36448437, 0.3422735 , 0.576     ],)
([0.40450853, 0.29389256, 0.576     ],)
([0.36448437, 0.3422735 , 0.564     ],)
([0.36448437, 0.3422735 , 0.564     ],)
([0.31871206, 0.38525656, 0.564     ],)
([0.36448437, 0.3422735 , 0.576     ],)
([0.31871206, 0.38525656, 0.576     ],)
([0.36448437, 0.3422735 , 0.576     ],)
([0.31871206, 0.38525656, 0.564     ],)
([0.31871206, 0.38525656, 0.564     ],)
([0.2679135, 0.4221639, 0.564    ],)
([0.31871206, 0.38525656, 0.576     ],)
([0.2679135, 0.4221639, 0.576    ],)
([0.31871206, 0.38525656, 0.576     ],)
([0.2679135, 0.4221639, 0.564    ],)
([0.2679135, 0.4221639, 0.564    ],)
([0.21288975, 0.45241347, 0.564     ],)
([0.2679135, 0.4221639, 0.576    ],)
([0.21288975, 0.45241347, 0.576     ],)
([0.2679135, 0.

([-0.15450893, -0.47552812,  0.588     ],)
([-0.09369112, -0.49114352,  0.588     ],)
([-0.15450893, -0.47552812,  0.588     ],)
([-0.09369112, -0.49114352,  0.576     ],)
([-0.09369112, -0.49114352,  0.576     ],)
([-0.03139574, -0.49901333,  0.576     ],)
([-0.09369112, -0.49114352,  0.588     ],)
([-0.03139574, -0.49901333,  0.588     ],)
([-0.09369112, -0.49114352,  0.588     ],)
([-0.03139574, -0.49901333,  0.576     ],)
([-0.03139574, -0.49901333,  0.576     ],)
([ 0.03139476, -0.4990134 ,  0.576     ],)
([-0.03139574, -0.49901333,  0.588     ],)
([ 0.03139476, -0.4990134 ,  0.588     ],)
([-0.03139574, -0.49901333,  0.588     ],)
([ 0.03139476, -0.4990134 ,  0.576     ],)
([ 0.03139476, -0.4990134 ,  0.576     ],)
([ 0.09369016, -0.49114373,  0.576     ],)
([ 0.03139476, -0.4990134 ,  0.588     ],)
([ 0.09369016, -0.49114373,  0.588     ],)
([ 0.03139476, -0.4990134 ,  0.588     ],)
([ 0.09369016, -0.49114373,  0.576     ],)
([ 0.09369016, -0.49114373,  0.576     ],)
([ 0.154508

([ 0.43815306, -0.24087736,  0.6       ],)
([0. , 0. , 0.6],)
([ 0.43815306, -0.24087736,  0.588     ],)
([ 0.464888  , -0.18406285,  0.588     ],)
([ 0.43815306, -0.24087736,  0.6       ],)
([ 0.464888  , -0.18406285,  0.6       ],)
([ 0.43815306, -0.24087736,  0.6       ],)
([ 0.464888  , -0.18406285,  0.588     ],)
([ 0.43815306, -0.24087736,  0.6       ],)
([ 0.464888  , -0.18406285,  0.6       ],)
([0. , 0. , 0.6],)
([ 0.464888  , -0.18406285,  0.588     ],)
([ 0.48429143, -0.12434555,  0.588     ],)
([ 0.464888  , -0.18406285,  0.6       ],)
([ 0.48429143, -0.12434555,  0.6       ],)
([ 0.464888  , -0.18406285,  0.6       ],)
([ 0.48429143, -0.12434555,  0.588     ],)
([ 0.464888  , -0.18406285,  0.6       ],)
([ 0.48429143, -0.12434555,  0.6       ],)
([0. , 0. , 0.6],)
([ 0.48429143, -0.12434555,  0.588     ],)
([ 0.49605727, -0.06266725,  0.588     ],)
([ 0.48429143, -0.12434555,  0.6       ],)
([ 0.49605727, -0.06266725,  0.6       ],)
([ 0.48429143, -0.12434555,  0.6       ]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [13]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Abaixo, enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [14]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [15]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [16]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [17]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Pegamos a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela.

In [18]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, exibimos a janela.

In [19]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.


Usaremos o GL_TRIANGLE para modelar os triângulos (que formarão outros polígonos) da superfície da nossa esfera.

In [20]:
import math
import random
d = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    ### apenas para visualizarmos a esfera rotacionando
    d -= 0.01 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_d,  0.0, sin_d, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_d, 0.0, cos_d, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_x)
    mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
    for triangle in range(0,len(vertices),3):
       
        random.seed( triangle )
        R = random.random()
        G = random.random()
        B = random.random()  
        glUniform4f(loc_color, R, G, B, 1.0)
        
        glDrawArrays(GL_TRIANGLES, triangle, 3)     


    
    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

Modifique esse código para aplicar outras transformações (de sua escolha) conforme eventos do teclado.